In [1]:
!pip install datasets accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found 

In [2]:
TRAIN_PATH = r'/kaggle/input/raw-dataset/vlsp2018_restaurant/1-VLSP2018-SA-Restaurant-train.csv'
VAL_PATH = r'/kaggle/input/raw-dataset/vlsp2018_restaurant/2-VLSP2018-SA-Restaurant-dev.csv'
TEST_PATH = r'/kaggle/input/raw-dataset/vlsp2018_restaurant/3-VLSP2018-SA-Restaurant-test.csv'
PRETRAINED_MODEL = 'vinai/phobert-base'
MODEL_NAME = 'Restaurant-v2' # Multitask model on Restaurant domain
MAX_LENGTH = 512
BATCH_SIZE = 21
EPOCHS = 20

In [3]:
from datasets import load_dataset

def load(train_csv_path, val_csv_path, test_csv_path):
        dataset_paths = {'train': train_csv_path, 'val': val_csv_path, 'test': test_csv_path}
        raw_datasets = load_dataset('csv', data_files={ k: v for k, v in dataset_paths.items() if v })
        return raw_datasets

In [4]:
raw_datasets = load(TRAIN_PATH, VAL_PATH, TEST_PATH)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
raw_datasets['train'][0]

{'Review': '_ Ảnh chụp từ hôm qua, đi chơi với gia đình và 1 nhà họ hàng đang sống tại Sài Gòn. _ Hôm qua đi ăn trưa muộn, ai cũng đói hết nên lúc có đồ ăn là nhào vô ăn liền, bởi vậy mới quên chụp các phần gọi thêm với nước mắm, chỉ chụp món chính thôi! _ Đói quá nên không biết đánh giá đồ ăn kiểu gì luôn 😅😅😅_ Chọn cái này vì thấy nó lạ với tui.',
 'AMBIENCE#GENERAL': 0,
 'DRINKS#PRICES': 0,
 'DRINKS#QUALITY': 0,
 'DRINKS#STYLE&OPTIONS': 0,
 'FOOD#PRICES': 0,
 'FOOD#QUALITY': 3,
 'FOOD#STYLE&OPTIONS': 3,
 'LOCATION#GENERAL': 0,
 'RESTAURANT#GENERAL': 0,
 'RESTAURANT#MISCELLANEOUS': 0,
 'RESTAURANT#PRICES': 0,
 'SERVICE#GENERAL': 0}

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
tokens = tokenizer.encode('Tôi là sinh_viên trường đại_học Công_nghệ thông_tin .')
print('Encode:', tokens, '\nDecode:', tokenizer.decode(tokens))
tokenizer.model_input_names

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Encode: [0, 218, 8, 649, 212, 956, 2413, 195, 5, 2] 
Decode: <s> Tôi là sinh_viên trường đại_học Công_nghệ thông_tin . </s>


['input_ids', 'token_type_ids', 'attention_mask']

In [7]:
# def labels_to_flatten_onehot(datasets):
#         print('[INFO] Transforming "Aspect#Categoy,Polarity" labels to flattened one-hot encoding...')
#         model_input_names = ['input_ids', 'token_type_ids', 'attention_mask']
#         label_columns = [col for col in datasets['train'].column_names if col not in ['Review', *model_input_names]]
#         def transform_each_review(review): # Convert each Aspect#Categoy,Polarity to one-hot encoding and merge them into 1D list
#             review['FlattenOneHotLabels'] = sum([
#                 PolarityMapping.INDEX_TO_ONEHOT[review[aspect_category]] # Get one-hot encoding
#                 for aspect_category in label_columns
#             ], []) # Need to be flattened to match the model's output shape
#             return review 
#         return datasets.map(transform_each_review, num_proc=8).select_columns(['FlattenOneHotLabels', *model_input_names])

def labels_to_flatten_onehot(datasets):
    print('[INFO] Transforming "Aspect#Categoy,Polarity" labels to flattened one-hot encoding...')
    model_input_names = ['input_ids', 'token_type_ids', 'attention_mask']
    label_columns = [col for col in datasets['train'].column_names if col not in ['Review', *model_input_names]]
    def transform_each_review(review):
        review['labels'] = sum([
            PolarityMapping.INDEX_TO_ONEHOT[review[aspect_category]]
            for aspect_category in label_columns
        ], [])
        return review 
    return datasets.map(transform_each_review, num_proc=8).select_columns(['labels', *model_input_names])

In [8]:
class PolarityMapping:
    INDEX_TO_POLARITY = { 0: None, 1: 'positive', 2: 'negative', 3: 'neutral' }
    # INDEX_TO_ONEHOT = { i: [1 if i == j else 0 for j in INDEX_TO_POLARITY] for i in INDEX_TO_POLARITY } 
    # POLARITY_TO_INDEX = { polarity: index for index, polarity in INDEX_TO_POLARITY.items() }
    INDEX_TO_ONEHOT = { 0: [1, 0, 0, 0], 1: [0, 1, 0, 0], 2: [0, 0, 1, 0], 3: [0, 0, 0, 1] }
    POLARITY_TO_INDEX = { None: 0, 'positive': 1, 'negative': 2, 'neutral': 3 }

In [9]:
from datasets import load_from_disk
# preprocessed_datasets = load_from_disk('/kaggle/input/preprocessed-restaurant/preprocessed_restaurant')
# preprocessed_datasets = labels_to_flatten_onehot(preprocessed_datasets)

import shutil

# Copy thư mục sang khu vực có thể ghi
shutil.copytree('/kaggle/input/preprocessed-restaurant/preprocessed_restaurant', 
                '/kaggle/working/preprocessed_restaurant')

# Load lại từ thư mục có thể ghi
preprocessed_datasets = load_from_disk('/kaggle/working/preprocessed_restaurant')

# Gọi map như bình thường
preprocessed_datasets = labels_to_flatten_onehot(preprocessed_datasets)

[INFO] Transforming "Aspect#Categoy,Polarity" labels to flattened one-hot encoding...


Map (num_proc=8):   0%|          | 0/2961 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1290 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
preprocessed_datasets['train']['labels'][0]

[1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0]

In [11]:
# def cast_labels_to_float(dataset):
#     def cast(example):
#         example['labels'] = [float(x) for x in example['labels']]
#         return example
#     return dataset.map(cast)

# preprocessed_datasets = cast_labels_to_float(preprocessed_datasets)

from datasets import Sequence, Value

num_labels = len(preprocessed_datasets['train'][0]['labels'])
for split in preprocessed_datasets:
    preprocessed_datasets[split] = preprocessed_datasets[split].cast_column(
        "labels", Sequence(feature=Value("float32"), length=num_labels)
    )

Casting the dataset:   0%|          | 0/2961 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1290 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

In [12]:
preprocessed_datasets['train'][0]['labels'][0]

1.0

In [13]:
import numpy as np
print('Aspect#Category,Polarity in One-hot form of the first review:')
example_onehot = preprocessed_datasets['train'][0]['labels']
np.array(example_onehot).reshape(-1, 4)

Aspect#Category,Polarity in One-hot form of the first review:


array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [14]:
ASPECT_CATEGORY_NAMES = raw_datasets['train'].column_names[1:]
steps_per_epoch = len(preprocessed_datasets['train']) // BATCH_SIZE
total_steps = EPOCHS * steps_per_epoch
print(ASPECT_CATEGORY_NAMES)

['AMBIENCE#GENERAL', 'DRINKS#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE&OPTIONS', 'FOOD#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE&OPTIONS', 'LOCATION#GENERAL', 'RESTAURANT#GENERAL', 'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#PRICES', 'SERVICE#GENERAL']


In [15]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, AutoModelForSequenceClassification

num_labels = len(preprocessed_datasets['train'][0]['labels'])
model = AutoModelForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL,
    num_labels=num_labels,
    problem_type="multi_label_classification"
)

2025-05-27 15:37:36.384249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748360256.577670      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748360256.633565      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results/output",
    learning_rate=1e-4,  # Initial learning rate
    num_train_epochs=10,  # Max number of epochs
    per_device_train_batch_size=16,
    eval_strategy="steps",  # Evaluate every few steps
    eval_steps=500,
    logging_steps=500,
    save_steps=1000,
    warmup_steps=int(total_steps * 0.15),  # Warmup phase for learning rate
    weight_decay=0.01,  # AdamW weight decay
    save_total_limit=3,  # Keep only 3 model checkpoints
    load_best_model_at_end=True,  # Restore best weights
    metric_for_best_model="eval_loss",  # Monitor validation loss
    greater_is_better=False,  # Lower validation loss is better
    lr_scheduler_type="cosine",  # Use cosine scheduler here
    report_to="none"
)

# Define EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3  # Stop training if no improvement in 3 evaluation rounds
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_datasets['train'],
    eval_dataset=preprocessed_datasets['val'],
    tokenizer=tokenizer,
    callbacks=[early_stopping]
)

# Start training
trainer.train()

/tmp/ipykernel_35/3831133099.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Step,Training Loss,Validation Loss
500,0.308400,0.189795
1000,0.168500,0.156417
1500,0.110900,0.148565


TrainOutput(global_step=1860, training_loss=0.1755090118736349, metrics={'train_runtime': 852.7771, 'train_samples_per_second': 34.722, 'train_steps_per_second': 2.181, 'total_flos': 3896968018083840.0, 'train_loss': 0.1755090118736349, 'epoch': 10.0})

In [17]:
def argmax_label_matrix(label_matrix):
    if not isinstance(label_matrix, np.ndarray): label_matrix = np.array(label_matrix)
    y = label_matrix.reshape(len(label_matrix), -1, 4)
    return np.argmax(y, axis=-1)

In [18]:
predictions = trainer.predict(preprocessed_datasets['test'])
predictions.predictions[0]

array([ 2.955475  , -2.7766767 , -2.3429182 , -2.7457082 , -0.40474316,
       -0.3862121 , -2.6265047 , -1.2095242 , -0.73370165,  0.7121926 ,
       -2.1297956 , -2.1478033 , -0.7150333 ,  0.79161656, -2.342866  ,
       -2.519134  ,  1.1810652 , -1.5831907 , -2.6027539 , -1.2902763 ,
       -0.93764806,  1.1242148 , -2.6170385 , -2.4766338 , -1.7793263 ,
        1.3195544 , -2.5196817 , -1.7629948 ,  1.4034461 , -1.626539  ,
       -2.2587843 , -2.3160217 , -1.2381275 ,  1.1871612 , -2.4933667 ,
       -2.7503474 ,  0.9051593 , -1.8743943 , -2.2056801 , -1.6836153 ,
        0.765071  , -0.8530801 , -2.3879147 , -1.8780257 ,  1.3164442 ,
       -1.2808921 , -2.4383397 , -2.3295615 ], dtype=float32)

In [19]:
y_pred = argmax_label_matrix(predictions.predictions)
y_pred[10]

array([1, 0, 0, 0, 3, 1, 1, 3, 1, 0, 0, 0])

In [20]:
y_test = argmax_label_matrix(preprocessed_datasets['test']['labels'])
y_test[10]

array([0, 0, 0, 0, 3, 1, 1, 3, 0, 0, 0, 0])

In [21]:
import sys
import pandas as pd
from tqdm import tqdm
from IPython.display import display_html
from sklearn.metrics import classification_report
sys.path.append('..')


class VLSP2018SklearnEvaluator:
    def __init__(self, y_test, y_pred, aspect_category_names): 
        aspect_cate_test, aspect_cate_pred = [], []
        aspect_cate_polar_test, aspect_cate_polar_pred = [], []

        for row_test, row_pred in zip(y_test, y_pred):
            for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
                aspect_cate_test.append(aspect_category_names[index] if col_test != 0 else 'None#None')
                aspect_cate_pred.append(aspect_category_names[index] if col_pred != 0 else 'None#None')
                aspect_cate_polar_test.append(aspect_category_names[index] + f',{PolarityMapping.INDEX_TO_POLARITY[col_test]}')
                aspect_cate_polar_pred.append(aspect_category_names[index] + f',{PolarityMapping.INDEX_TO_POLARITY[col_pred]}')

        self.aspect_cate_polar_report = classification_report(aspect_cate_polar_test, aspect_cate_polar_pred, output_dict=True, zero_division=1.0)
        self.aspect_cate_report = classification_report(aspect_cate_test, aspect_cate_pred, output_dict=True, zero_division=1.0)
        self.polarity_report = classification_report(y_test.flatten(), y_pred.flatten(), target_names=PolarityMapping.POLARITY_TO_INDEX, output_dict=True)
        self._merge_all_reports()
        self._build_macro_avg_df()
        
    
    def report(self, report_type='all'):
        if report_type.lower() == 'all': self._display_all_reports()
        elif report_type.lower() == 'aspect#category,polarity': return pd.DataFrame(self.aspect_cate_polar_report).T
        elif report_type.lower() == 'aspect#category': return pd.DataFrame(self.aspect_cate_report).T
        elif report_type.lower() == 'polarity': return pd.DataFrame(self.polarity_report).T
        elif report_type.lower() == 'macro_avg': return self.macro_avg_df()
        else: raise ValueError('report_type must be in ["all", "aspect#category,polarity", "aspect#category", "polarity", "macro_avg"]')
        
        
    def _merge_all_reports(self):
        self.merged_report = {}
        for key, metrics in self.aspect_cate_polar_report.items():
            # Check if key in the form of 'aspect#category,polarity' (Check if it's not 'accuracy' or 'macro avg' or 'weighted avg')
            if key in ['accuracy', 'macro avg', 'weighted avg']:
                self.merged_report[key] = {
                    'aspect#category': self.aspect_cate_report[key],
                    'aspect#category,polarity': metrics
                }
            else:
                aspect_cate, polarity = key.split(',')
                if aspect_cate not in self.merged_report:
                    self.merged_report[aspect_cate] = {'aspect#category': self.aspect_cate_report[aspect_cate]}
                self.merged_report[aspect_cate][polarity] = metrics
                
                
    def _build_macro_avg_df(self):
        self.macro_avg_df = pd.DataFrame([{
            'accuracy': f"{report['accuracy']:.3f}", 
            # **{metric: report['macro avg'][metric] for metric in report['macro avg'] if metric != 'accuracy'}
            'precision': f"{report['macro avg']['precision']:.3f}",
            'recall': f"{report['macro avg']['recall']:.3f}",
            'f1-score': f"{report['macro avg']['f1-score']:.3f}",
            'support': report['macro avg']['support']
        } for report in [self.aspect_cate_polar_report, self.aspect_cate_report, self.polarity_report]])
        self.macro_avg_df.index = ['Aspect#Category,Polarity', 'Aspect#Category', 'Polarity']
    
        
    def _display_all_reports(self):
        metric_names = list(self.merged_report.values())[0]['aspect#category']
        html_str = f"""
            <tr>
                <th style="font-weight: bold; text-align: center;" rowspan="2">ACSA Report (w/o "None" polarity)</th>
                <th style="font-weight: bold; text-align: center;" colspan="{len(metric_names)}">Aspect#Category</th>
                <th style="font-weight: bold; text-align: center;" colspan="{len(metric_names)}">Aspect#Category,Polarity</th>
            </tr>
            <tr>
                {''.join([f'<th>{metric_name}</th>' for metric_name in metric_names] * 2)}
            </tr>
        """

        for key, merged_metrics in tqdm(self.merged_report.items()):
            if key in ['accuracy', 'macro avg', 'weighted avg']: continue
            polarities = merged_metrics.keys() - {'aspect#category', 'None'}
            aspect_cate_html = ''.join(
                f'<td rowspan="{len(polarities)}">{value if name == "support" else f"{value:.3f}"}</td>'
                for name, value in self.merged_report[key]['aspect#category'].items()
            )
            for index, polarity in enumerate(polarities):
                aspect_cate_polar_html = ''.join(
                    f'<td>{value if name == "support" else f"{value:.3f}"}</td>'
                    for name, value in self.merged_report[key][polarity].items()
                )
                html_str += f"""
                    <tr>
                        <td>{key},{polarity}</td>
                        {aspect_cate_html if index == 0 else ''}
                        {aspect_cate_polar_html}
                    </tr>
                """

        display_html(f'''
            <div style="display: flex; align-items: flex-start; flex-wrap: nowrap">
                <table style="margin-right: 10px">{html_str}</table> 
                <div style="display: flex; align-items: center; flex-direction: column">
                    <b>Polarity Report</b><br>
                    {pd.DataFrame(self.polarity_report).T.to_html()}<br>
                    <b>Macro Avg Report</b><br>
                    {self.macro_avg_df.to_html()}
                </div>
            </div>
        ''', raw=True)      

In [22]:
sk_eval = VLSP2018SklearnEvaluator(y_test, y_pred, ASPECT_CATEGORY_NAMES)
# sk_eval.report(report_type='Aspect#Category,Polarity').round(3)
# sk_eval.report(report_type='Aspect#Category').round(3)
# sk_eval.report(report_type='Polarity').round(3)
# sk_eval.report(report_type='macro_avg').round(3)
sk_eval.report(report_type='all')

100%|██████████| 15/15 [00:00<00:00, 64793.57it/s]


ACSA Report (w/o "None" polarity) 
 Aspect#Category 
 Aspect#Category,Polarity 
 
 
 precision recall f1-score support precision recall f1-score support 
 
 
 
 AMBIENCE#GENERAL,neutral 
 0.885 0.816 0.849 255 
 0.333 0.083 0.133 48 
 
 
 
 AMBIENCE#GENERAL,positive 
 
 0.700 0.862 0.772 181 
 
 
 
 AMBIENCE#GENERAL,negative 
 
 1.000 0.000 0.000 26 
 
 
 
 DRINKS#PRICES,neutral 
 0.857 0.079 0.145 76 
 1.000 0.000 0.000 45 
 
 
 
 DRINKS#PRICES,positive 
 
 0.143 0.036 0.057 28 
 
 
 
 DRINKS#PRICES,negative 
 
 1.000 0.000 0.000 3 
 
 
 
 DRINKS#QUALITY,neutral 
 1.000 0.169 0.289 71 
 1.000 0.000 0.000 11 
 
 
 
 DRINKS#QUALITY,positive 
 
 0.917 0.204 0.333 54 
 
 
 
 DRINKS#QUALITY,negative 
 
 1.000 0.000 0.000 6 
 
 
 
 DRINKS#STYLE&OPTIONS,neutral 
 0.789 0.326 0.462 46 
 1.000 0.000 0.000 4 
 
 
 
 DRINKS#STYLE&OPTIONS,positive 
 
 0.632 0.293 0.400 41 
 
 
 
 DRINKS#STYLE&OPTIONS,negative 
 
 1.000 0.000 0.000 1 
 
 
 
 FOOD#PRICES,neutral 
 0.837 0.946 0.888 331 
 0.702 0.646 0.673 175 
 
 
 
 FOOD#PRICES,positive 
 
 0.505 0.836 0.629 128 
 
 
 
 FOOD#PRICES,negative 
 
 1.000 0.036 0.069 28 
 
 
 
 FOOD#QUALITY,neutral 
 0.914 1.000 0.955 457 
 0.556 0.116 0.192 43 
 
 
 
 FOOD#QUALITY,positive 
 
 0.821 1.000 0.902 403 
 
 
 
 FOOD#QUALITY,negative 
 
 1.000 0.000 0.000 11 
 
 
 
 FOOD#STYLE&OPTIONS,neutral 
 0.885 0.953 0.918 403 
 1.000 0.000 0.000 53 
 
 
 
 FOOD#STYLE&OPTIONS,positive 
 
 0.749 0.973 0.846 334 
 
 
 
 FOOD#STYLE&OPTIONS,negative 
 
 1.000 0.000 0.000 16 
 
 
 
 LOCATION#GENERAL,neutral 
 0.877 0.715 0.788 179 
 0.531 0.768 0.628 99 
 
 
 
 LOCATION#GENERAL,positive 
 
 0.000 0.000 0.000 64 
 
 
 
 LOCATION#GENERAL,negative 
 
 1.000 0.000 0.000 16 
 
 
 
 RESTAURANT#GENERAL,neutral 
 0.632 0.807 0.709 223 
 1.000 0.000 0.000 5 
 
 
 
 RESTAURANT#GENERAL,positive 
 
 0.596 0.829 0.694 205 
 
 
 
 RESTAURANT#GENERAL,negative 
 
 1.000 0.000 0.000 13 
 
 
 
 RESTAURANT#MISCELLANEOUS,neutral 
 1.000 0.000 0.000 130 
 1.000 0.000 0.000 62 
 
 
 
 RESTAURANT#MISCELLANEOUS,positive 
 
 1.000 0.000 0.000 59 
 
 
 
 RESTAURANT#MISCELLANEOUS,negative 
 
 1.000 0.000 0.000 9 
 
 
 
 RESTAURANT#PRICES,neutral 
 1.000 0.000 0.000 73 
 1.000 0.000 0.000 24 
 
 
 
 RESTAURANT#PRICES,positive 
 
 1.000 0.000 0.000 44 
 
 
 
 RESTAURANT#PRICES,negative 
 
 1.000 0.000 0.000 5 
 
 
 
 SERVICE#GENERAL,neutral 
 0.777 0.777 0.777 175 
 1.000 0.000 0.000 22 
 
 
 
 SERVICE#GENERAL,positive 
 
 0.573 0.766 0.656 128 
 
 
 
 SERVICE#GENERAL,negative 
 
 1.000 0.160 0.276 25 
 
 
 
 Polarity Report 
 
 
 
 
 precision 
 recall 
 f1-score 
 support 
 
 
 
 
 None 
 0.847889 
 0.902820 
 0.874493 
 3581.0000 
 
 
 positive 
 0.690899 
 0.768724 
 0.727737 
 1669.0000 
 
 
 negative 
 1.000000 
 0.031447 
 0.060976 
 159.0000 
 
 
 neutral 
 0.609231 
 0.335025 
 0.432314 
 591.0000 
 
 
 accuracy 
 0.786500 
 0.786500 
 0.786500 
 0.7865 
 
 
 macro avg 
 0.787005 
 0.509504 
 0.523880 
 6000.0000 
 
 
 weighted avg 
 0.784743 
 0.786500 
 0.768557 
 6000.0000 
 
 
 
 Macro Avg Report 
 
 
 
 
 accuracy 
 precision 
 recall 
 f1-score 
 support 
 
 
 
 
 Aspect#Category,Polarity 
 0.786 
 0.832 
 0.355 
 0.344 
 6000 
 
 
 Aspect#Category 
 0.845 
 0.869 
 0.576 
 0.589 
 6000 
 
 
 Polarity 
 0.786 
 0.787 
 0.510 
 0.524 
 6000

In [23]:
def vlsp_save_as(save_path, raw_texts, encoded_review_labels, aspect_category_names):
        with open(save_path, 'w', encoding='utf-8') as file:
            for index, encoded_label in tqdm(enumerate(encoded_review_labels)):
                polarities = map(lambda x: PolarityMapping.INDEX_TO_POLARITY[x], encoded_label)
                acsa = ', '.join(
                    f'{{{aspect_category}, {polarity}}}' 
                    for aspect_category, polarity in zip(aspect_category_names, polarities) if polarity
                )
                file.write(f"#{index + 1}\n{raw_texts[index]}\n{acsa}\n\n")

In [24]:
vlsp_save_as(f'/kaggle/working/{MODEL_NAME}.txt', raw_datasets['test']['Review'], y_pred, ASPECT_CATEGORY_NAMES)

500it [00:00, 86334.53it/s]


In [25]:
!java /kaggle/input/sa-evaluate/SAEvaluate.java /kaggle/input/raw-dataset/vlsp2018_restaurant/3-VLSP2018-SA-Restaurant-test.txt /kaggle/working/{MODEL_NAME}.txt

Row mismatch:﻿#1 <-> #1
Evaluation Result >> File:/kaggle/working/Restaurant-v2.txt<> [/kaggle/input/raw-dataset/vlsp2018_restaurant/3-VLSP2018-SA-Restaurant-test.txt]
                    Gold count	223	71	76	46	179	130	403	331	457	255	73	175
                  ANSWER count	285	12	7	19	146	0	434	374	500	235	0	175

        Correct ANSWER: aspect	179	11	5	14	128	0	384	313	457	208	0	136
             Precision: aspect	0.6281	0.9167	0.7143	0.7368	0.8767	0.0000	0.8848	0.8369	0.9140	0.8851	0.0000	0.7771
                Recall: aspect	0.8027	0.1549	0.0658	0.3043	0.7151	0.0000	0.9529	0.9456	1.0000	0.8157	0.0000	0.7771
              F1 score: aspect	0.7047	0.2651	0.1205	0.4308	0.7877	0.0000	0.9176	0.8879	0.9551	0.8490	0.0000	0.7771

  Over All ANSWER: aspect:----	0.8390	0.7586	0.7968

  Correct ANSWER: aspect,value	169	10	1	12	76	0	325	221	408	160	0	102
      Precision: aspect, value	0.5930	0.8333	0.1429	0.6316	0.5205	0.0000	0.7488	0.5909	0.8160	0.6809	0.0000	0.5829
         Recall: aspect, value

---------------------------------------------------------------


In [26]:
output = "/kaggle/working/savedmodel"
model.save_pretrained(output)
tokenizer.save_pretrained(output)

('/kaggle/working/savedmodel/tokenizer_config.json',
 '/kaggle/working/savedmodel/special_tokens_map.json',
 '/kaggle/working/savedmodel/vocab.txt',
 '/kaggle/working/savedmodel/bpe.codes',
 '/kaggle/working/savedmodel/added_tokens.json')